In [1]:
!pip install ipydeps

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 2.3MB 8.0MB/s 
  Created wheel for ipydeps: filename=ipydeps-0.10.0-cp36-none-any.whl size=10798 sha256=1c07a7a460ddf91ae05c57383f96f7d0be125d98642f1f4de4477771a039298f
  Stored in directory: /root/.cache/pip/wheels/e3/4a/6f/2a1810efd60598edd3f3a3d800bb46f734407dd62c18710534
  Created wheel for pypki2: filename=pypki2-0.11.0-cp36-none-any.whl size=10428 sha256=62bd298da4fd0f157027984d0c7675c58f9253d2dd6414f811ebcf3fde643907
  Stored in directory: /root/.cache/pip/wheels/94/40/0b/348ab8c4a7ccd1f846f0b7a011065f70b06a53c7accf84a301
Successfully built ipydeps pypki2


In [2]:
import ipydeps
ipydeps.pip(["keras-rl", "gym"])

In [3]:
!pip uninstall tensorflow==2.0.0-beta

Uninstalling tensorflow-1.15.0:
  Would remove:
    /tensorflow-1.15.0/python3.6/tensorflow-1.15.0.dist-info/*
    /tensorflow-1.15.0/python3.6/tensorflow/*
    /tensorflow-1.15.0/python3.6/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0


In [4]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 103kB/s 
     |████████████████████████████████| 3.2MB 46.4MB/s 
     |████████████████████████████████| 368kB 38.4MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.13.1 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 2.1.0
    Uninstalling tensorflow-2.1.0:
      Successfully uninstalled tensorflow-2.1.0


In [5]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1


In [6]:
import numpy as np
import gym
from gym import spaces
from gym.utils import seeding

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.


In [0]:
import copy
import random
def cmp(a, b):
    return float(a > b) - float(a < b)

# 1 = Ace, 2-10 = Number cards, Jack/Queen/King = 10
deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]
deck2 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]
countn = 0


def draw_card(rand):
    global countn
    curr = deck[rand]
    deck.remove(curr)
    if int(curr) == 1 or int(curr) == 10:
        countn -= 1
    if int(curr) < 7:
        countn += 1
    return int(curr)


def draw_hand():
    return [draw_card(random.randint(0, len(deck)-1)), draw_card(random.randint(0, len(deck)-1))]


def usable_ace(hand):  # Does this hand have a usable ace?
    return 1 in hand and sum(hand) + 10 <= 21


def sum_hand(hand):  # Return current hand total
    if usable_ace(hand):
        return sum(hand) + 10
    return sum(hand)


def is_bust(hand):  # Is this hand a bust?
    return sum_hand(hand) > 21


def score(hand):  # What is the score of this hand (0 if bust)
    return 0 if is_bust(hand) else sum_hand(hand)


def is_natural(hand):  # Is this hand a natural blackjack?
    return sorted(hand) == [1, 10]


class CountBlackjackEnv(gym.Env):
    """Simple blackjack environment
    Blackjack is a card game where the goal is to obtain cards that sum to as
    near as possible to 21 without going over.  They're playing against a fixed
    dealer.
    Face cards (Jack, Queen, King) have point value 10.
    Aces can either count as 11 or 1, and it's called 'usable' at 11.
    This game is placed with an infinite deck (or with replacement).
    The game starts with each (player and dealer) having one face up and one
    face down card.
    The player can request additional cards (hit=1) until they decide to stop
    (stick=0) or exceed 21 (bust).
    After the player sticks, the dealer reveals their facedown card, and draws
    until their sum is 17 or greater.  If the dealer goes bust the player wins.
    If neither player nor dealer busts, the outcome (win, lose, draw) is
    decided by whose sum is closer to 21.  The reward for winning is +1,
    drawing is 0, and losing is -1.
    The observation of a 3-tuple of: the players current sum,
    the dealer's one showing card (1-10 where 1 is ace),
    and whether or not the player holds a usable ace (0 or 1).
    This environment corresponds to the version of the blackjack problem
    described in Example 5.1 in Reinforcement Learning: An Introduction
    by Sutton and Barto.
    http://incompleteideas.net/book/the-book-2nd.html
    """
    def __init__(self, natural=False):
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Tuple((
            spaces.Discrete(32),
            spaces.Discrete(40),
            spaces.Discrete(2)))

        # Flag to payout 1.5 on a "natural" blackjack win, like casino rules
        # Ref: http://www.bicyclecards.com/how-to-play/blackjack/
        self.natural = natural
        # Start the first game
        self.reset()


    def step(self, action):
        global deck
        global countn
        assert self.action_space.contains(action)
        if action:  # hit: add a card to players hand and return
            if len(deck) == 0:
                deck = copy.copy(deck2)
                countn = 0
            self.player.append(draw_card(random.randint(0, len(deck)-1)))
            if is_bust(self.player):
                done = True
                reward = -1
            else:
                done = False
                reward = 0
        else:  # stick: play out the dealers hand, and score
            done = True
            while sum_hand(self.dealer) < 17:
                if len(deck) == 0:
                    deck = copy.copy(deck2)
                    countn = 0
                self.dealer.append(draw_card(random.randint(0, len(deck)-1)))
            reward = cmp(score(self.player), score(self.dealer))
            if self.natural and is_natural(self.player) and reward == 1:
                reward = 1.5
        return self._get_obs(), reward, done, {}

    def _get_obs(self):
        return (sum_hand(self.player), countn, usable_ace(self.player))

    def reset(self):
        global deck
        global countn
        if len(deck) < 4:
            deck = copy.copy(deck2)
            countn = 0
        self.dealer = draw_hand()
        self.player = draw_hand()
        return self._get_obs()

In [0]:
env = CountBlackjackEnv()
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [9]:
model = Sequential()
model.add(Flatten(input_shape=(1,3)))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                64        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
________________________________________________

In [0]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [11]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 10:38 - reward: 0.0000e+00WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


/root/.local/lib/python3.6/site-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 67s 7ms/step - reward: -0.1897
done, took 66.573 seconds


In [0]:
dqn.save_weights('dqn_countblackjack_v0_weights.h5f', overwrite=True)

In [18]:
dqn.test(env, nb_episodes=100, visualize=False)

Testing for 100 episodes ...
Episode 1: reward: 1.000, steps: 2
Episode 2: reward: -1.000, steps: 2
Episode 3: reward: -1.000, steps: 1
Episode 4: reward: -1.000, steps: 2
Episode 5: reward: -1.000, steps: 1
Episode 6: reward: 1.000, steps: 3
Episode 7: reward: -1.000, steps: 1
Episode 8: reward: -1.000, steps: 1
Episode 9: reward: -1.000, steps: 2
Episode 10: reward: -1.000, steps: 2
Episode 11: reward: 1.000, steps: 3
Episode 12: reward: -1.000, steps: 3
Episode 13: reward: 1.000, steps: 1
Episode 14: reward: -1.000, steps: 1
Episode 15: reward: -1.000, steps: 2
Episode 16: reward: 1.000, steps: 2
Episode 17: reward: 1.000, steps: 1
Episode 18: reward: -1.000, steps: 1
Episode 19: reward: -1.000, steps: 1
Episode 20: reward: 1.000, steps: 1
Episode 21: reward: -1.000, steps: 1
Episode 22: reward: -1.000, steps: 1
Episode 23: reward: -1.000, steps: 3
Episode 24: reward: 1.000, steps: 2
Episode 25: reward: 0.000, steps: 2
Episode 26: reward: 1.000, steps: 4
Episode 27: reward: -1.000, 

In [0]:
# Mr. Kyle King. 2020 March 6-7.Assistance given to the author, verbal discussion and electronic copying.
# https://gist.github.com/kylekyle/ba1d0d716b644e83495e95d68418167a Using this code and 
# verabl discussion Mr. King helped me understand how to use a reinforcement learning model
# and create an agent.

In [0]:
# Mr. Matthias Plappert. 2020 March 6. Electronic copying. https://github.com/keras-rl/keras-rl
# I used Keras-rl for this project and the example dqn_cartpole.py to understand how to import
# Keras featurs and compile my blackjack agent.

In [0]:
# Open AI gym contributers. 2020 March 6. Electronic copying. https://github.com/openai/gym/blob/master/gym/envs/toy_text/blackjack.py
# I modyfied the code from blackjack.py as my agent. Most of the methods from blackjack.py remain unchanged from the source.